https://www.thesportsdb.com/table.php?l=4334&s=2025-2026

In [12]:
import requests
import os
import json
import time
from dotenv import load_dotenv

In [13]:
url = "https://v3.football.api-sports.io/standings"
params = {
    "league": 61,  # Ligue 1
    "season": 2024
}

headers = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": API_HOST
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    standings = data['response'][0]['league']['standings'][0]

    # Transformer en DataFrame pour lecture facile
    df = pd.DataFrame([
        {
            "Position": team['rank'],
            "Équipe": team['team']['name'],
            "Points": team['points'],
            "Joués": team['all']['played'],
            "Gagnés": team['all']['win'],
            "Nuls": team['all']['draw'],
            "Perdus": team['all']['lose'],
            "Diff. Buts": team['goalsDiff']
        }
        for team in standings
    ])

    print(df)
else:
    print(f"Erreur {response.status_code}: {response.text}")

    Position               Équipe  Points  Joués  Gagnés  Nuls  Perdus  \
0          1  Paris Saint Germain      84     34      26     6       2   
1          2            Marseille      65     34      20     5       9   
2          3               Monaco      61     34      18     7       9   
3          4                 Nice      60     34      17     9       8   
4          5                Lille      60     34      17     9       8   
5          6                 Lyon      57     34      17     6      11   
6          7           Strasbourg      57     34      16     9       9   
7          8                 Lens      52     34      15     7      12   
8          9    Stade Brestois 29      50     34      15     5      14   
9         10             Toulouse      42     34      11     9      14   
10        11              Auxerre      42     34      11     9      14   
11        12               Rennes      41     34      13     2      19   
12        13               Nantes     

In [17]:
import requests
import os
import json
import time
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv()
API_KEY = os.getenv("API_FOOTBALL_KEY")
API_HOST = os.getenv("API_FOOTBALL_HOST")

# Paramètres de la ligue
LEAGUE_ID = 61   # Ligue 1
SEASON = 2024    # Saison 2024
CACHE_FILE = "standings_cache.json"
CACHE_DURATION = 24 * 3600  # 24 heures

def get_standings_from_api():
    """Récupère le classement depuis l'API-Football."""
    url = "https://v3.football.api-sports.io/standings"
    params = {"league": LEAGUE_ID, "season": SEASON}
    headers = {
        "x-rapidapi-key": API_KEY,
        "x-rapidapi-host": API_HOST
    }

    print("📡 Téléchargement du classement depuis l'API...")
    r = requests.get(url, headers=headers, params=params)
    r.raise_for_status()
    data = r.json()

    try:
        standings = data["response"][0]["league"]["standings"][0]
    except (KeyError, IndexError):
        raise ValueError("Aucune donnée de classement trouvée.")

    # Sauvegarde du cache
    with open(CACHE_FILE, "w", encoding="utf-8") as f:
        json.dump({"timestamp": time.time(), "standings": standings}, f, indent=2, ensure_ascii=False)

    return standings

def get_standings():
    """Charge le classement depuis le cache si valide, sinon via l’API."""
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE, "r", encoding="utf-8") as f:
                cache = json.load(f)
            age = time.time() - cache.get("timestamp", 0)
            if age < CACHE_DURATION and "standings" in cache:
                print("✅ Chargement du classement depuis le cache.")
                return cache["standings"]
            else:
                print("⚠️ Cache expiré ou invalide, téléchargement depuis l'API...")
        except (json.JSONDecodeError, KeyError):
            print("⚠️ Cache corrompu, nouveau téléchargement depuis l'API...")

    return get_standings_from_api()

def get_team_rank(team_name):
    """Récupère le rang d'une équipe dans le classement."""
    standings = get_standings()
    for team in standings:
        if team_name.lower() in team["team"]["name"].lower():
            return int(team["rank"])
    raise ValueError(f"Équipe '{team_name}' non trouvée dans la ligue.")

def proba_victoire(rang_A, rang_B):
    """Calcule les probabilités de victoire basées sur le classement."""
    pA = rang_B / (rang_A + rang_B)
    pB = rang_A / (rang_A + rang_B)
    return round(pA, 2), round(pB, 2)

def predire_match(equipe_A, equipe_B):
    """Prédit le vainqueur d’un match selon les rangs du classement."""
    try:
        rang_A = get_team_rank(equipe_A)
        rang_B = get_team_rank(equipe_B)
    except ValueError as e:
        print("Erreur :", e)
        return

    pA, pB = proba_victoire(rang_A, rang_B)
    prediction = equipe_A if pA > pB else equipe_B

    print(f"⚽ Match : {equipe_A} (#{rang_A}) vs {equipe_B} (#{rang_B})")
    print(f"Probabilité {equipe_A} : {pA*100}% | {equipe_B} : {pB*100}%")
    print(f"Cotes {equipe_A} : {round(1/pA, 2)} | {equipe_B} : {round(1/pB, 2)}")
    print(f"➡️ Pronostic : victoire de {prediction}")

# Exemple d'utilisation
predire_match("Auxerre", "Marseille")

⚠️ Cache corrompu, nouveau téléchargement depuis l'API...
📡 Téléchargement du classement depuis l'API...
✅ Chargement du classement depuis le cache.
⚽ Match : Auxerre (#11) vs Marseille (#2)
Probabilité Auxerre : 15.0% | Marseille : 85.0%
Cotes Auxerre : 6.67 | Marseille : 1.18
➡️ Pronostic : victoire de Marseille


In [18]:
# =====================================================
# 🔹 GESTION DU CLASSEMENT AVEC CACHE
# =====================================================

def get_standings_from_api():
    """Télécharge le classement depuis API-Football."""
    print("📡 Téléchargement du classement depuis l'API-Football...")
    url = "https://v3.football.api-sports.io/standings"
    params = {"league": LEAGUE_ID, "season": SEASON}
    headers = {
        "x-rapidapi-key": API_KEY,
        "x-rapidapi-host": API_HOST
    }

    r = requests.get(url, headers=headers, params=params)
    r.raise_for_status()
    data = r.json()

    try:
        standings = data["response"][0]["league"]["standings"][0]
    except (KeyError, IndexError):
        raise ValueError("Aucune donnée de classement trouvée pour cette saison.")

    # Sauvegarde du cache
    with open(CACHE_FILE, "w", encoding="utf-8") as f:
        json.dump({"timestamp": time.time(), "standings": standings}, f, indent=2, ensure_ascii=False)

    return standings

def get_standings():
    """Charge le classement depuis le cache si valide, sinon via l’API."""
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE, "r", encoding="utf-8") as f:
                cache = json.load(f)
            age = time.time() - cache.get("timestamp", 0)
            if age < CACHE_DURATION and "standings" in cache:
                print("✅ Chargement du classement depuis le cache.")
                return cache["standings"]
            else:
                print("⚠️ Cache expiré, rechargement depuis l'API...")
        except (json.JSONDecodeError, KeyError):
            print("⚠️ Cache corrompu, nouveau téléchargement depuis l'API...")

    return get_standings_from_api()

# =====================================================
# 🔹 ALGORITHME DE PRONOSTIC AVEC NUL
# =====================================================

def get_team_rank(team_name):
    """Récupère le rang d'une équipe dans le classement via API-Football."""
    standings = get_standings()
    for team in standings:
        if team_name.lower() in team["team"]["name"].lower():
            return int(team["rank"])
    raise ValueError(f"Équipe '{team_name}' non trouvée dans la ligue.")

def proba_victoire_avec_nul(rang_A, rang_B):
    """
    Calcule les probabilités de victoire et de nul selon le classement.
    """
    # Étape 1 : probabilités brutes selon les rangs
    pA = rang_B / (rang_A + rang_B)
    pB = rang_A / (rang_A + rang_B)

    # Étape 2 : nul plus probable si les équipes sont proches
    diff = abs(rang_A - rang_B)
    pN = 0.25 - (diff / 100)
    pN = max(0.05, min(pN, 0.30))  # bornes 5% à 30%

    # Étape 3 : normalisation pour que pA + pB + pN = 1
    total = pA + pB
    pA = pA / total * (1 - pN)
    pB = pB / total * (1 - pN)

    return round(pA, 3), round(pN, 3), round(pB, 3)

def pronostic(equipe_A, equipe_B):
    """Fait le pronostic entre deux équipes via l’API-Football."""
    try:
        rang_A = get_team_rank(equipe_A)
        rang_B = get_team_rank(equipe_B)
    except ValueError as e:
        print("Erreur :", e)
        return

    pA, pN, pB = proba_victoire_avec_nul(rang_A, rang_B)

    print(f"⚽ {equipe_A} (#{rang_A}) vs {equipe_B} (#{rang_B})")
    print(f"Probabilités : {equipe_A} {pA*100:.1f}% | Nul {pN*100:.1f}% | {equipe_B} {pB*100:.1f}%")
    print(f"Cotes : {equipe_A} {1/pA:.2f} | Nul {1/pN:.2f} | {equipe_B} {1/pB:.2f}")

    # Choix du pronostic
    if max(pA, pN, pB) == pA:
        prono = f"Victoire de {equipe_A}"
    elif max(pA, pN, pB) == pB:
        prono = f"Victoire de {equipe_B}"
    else:
        prono = "Match nul"

    print(f"➡️ Pronostic : {prono}\n")

# =====================================================
# 🧪 Exemple d’utilisation
# =====================================================
pronostic("Auxerre", "Marseille")

✅ Chargement du classement depuis le cache.
✅ Chargement du classement depuis le cache.
⚽ Auxerre (#11) vs Marseille (#2)
Probabilités : Auxerre 12.9% | Nul 16.0% | Marseille 71.1%
Cotes : Auxerre 7.75 | Nul 6.25 | Marseille 1.41
➡️ Pronostic : Victoire de Marseille

